## Hierarchical Architecture

In [11]:
from dotenv import load_dotenv
load_dotenv()

True

In [12]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts.chat import SystemMessage, _convert_to_message
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage, AIMessage
from langchain_core.output_parsers.openai_functions import JsonOutputFunctionsParser

from langgraph.graph import END, StateGraph, MessageGraph
from langgraph.checkpoint.sqlite import SqliteSaver

from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic

import functools
import operator
from typing import List, Sequence, TypedDict, Annotated
import json
import os
import random

from IPython.display import Image, display

import concurrent.futures

In [13]:
unique_id = "Hierarchical Optimisation"
os.environ["LANGCHAIN_PROJECT"] = f"Tracing Walkthrough - {unique_id}"

In [14]:
# from langsmith import Client

# client = Client()

In [15]:
class CorePrinciples:
    def __init__(self, core_principles: List[str]):
        self.core_principles = core_principles
    
    def add_principle(self, principle: str):
        """
        Adds a principle to the core principles list.
        
        :param principle: The principle to be added.
        """
        self.core_principles.append(principle)
        
    def __str__(self):
        """
        Returns a string representation of the core principles, each principle is listed on a new line with a preceding dash.
        
        Example:
        - principle 1
        - principle 2
        ...
        """
        return "\n".join([f"- {principle}" for principle in self.core_principles])

class WorkerAgent:
    """
    Worker Agent class defining agents that provide feedback on prompts.
    """

    def __init__(self, position: str, core_principles: CorePrinciples, llm):
        self.position = position
        self.core_principles = str(core_principles)
        self.system_message = f"""You are an experienced: {self.position}. Your core principles are:
{self.core_principles}"""      
        self.llm = llm

        assert isinstance(self.llm, ChatOpenAI) or isinstance(self.llm, ChatAnthropic), "The LLM must be an instance of ChatOpenAI or ChatAnthropic."

    def review_prompt(self, state: Sequence[BaseMessage], criteria: str):
        """
        Generates a review of the prompt.
        """
        prompt_text = f"""Your task is to provide feedback on the prompt in the conversation above in light of your core princples.
You must think outside the box and consider unconventional ideas.

The success criteria for the updated prompt are as follows:
{criteria}
You must use this information to inform your feedback.

Your reviewal process should be as follows:
1. Read the conversation carefully as an experienced {self.position}.
2. Explain how you think the prompt can improved in light of your core principles.
3. Submit your feedback.
"""
        if isinstance(self.llm, ChatOpenAI):
            prompt = ChatPromptTemplate.from_messages(
                [
                    ("system", self.system_message),
                    MessagesPlaceholder(variable_name="messages"),
                    ("system", prompt_text),
                ]
            )
        elif isinstance(self.llm, ChatAnthropic):
            prompt = ChatPromptTemplate.from_messages(
                [
                    ("system", self.system_message),
                    MessagesPlaceholder(variable_name="messages"),
                    ("user", prompt_text),
                ]
            )
        chain = prompt | self.llm
        result = chain.invoke({"messages": state})       
        return {"messages": [HumanMessage(content=result.content, name=self.position)]}

In [16]:
class TeamLeaderAgent:
    """
    TeamLeaderAgent class defining an agent that manages a team of worker agents to help optimise prompts.
    """

    def __init__(self, position: str, core_principles: CorePrinciples, prompt: str, criteria: str, llm, workforce: List[WorkerAgent]):
        self.position = position
        self.core_principles = str(core_principles)
        self.system_message = f"""You are an experienced: {self.position}. Your core principles are:
{self.core_principles}"""
        self.prompt = prompt
        self.criteria = criteria
        self.llm = llm
        self.workforce = workforce
        self.iterations = 0

    def reset(self):
        """
        Resets the agent state.
        """
        self.iterations = 0

    def leader_decision(self, state: Sequence[BaseMessage]):
        """
        LeaderAgent to decide the next team or to finish.
        """
        self.iterations += 1
        if self.iterations > 3:
            self.reset()
            return "FINISH"
        else:
            workers = ["FINISH"] + [worker.position for worker in self.workforce]
            workers_details = [f"{worker.position}:\n{worker.core_principles}" for worker in self.workforce]
            # shuffle the options to avoid positional bias
            random.shuffle(workers)
            # options = ["FINISH"] + positions
            function_def = {
                "name": "route",
                "description": "Select the next role.",
                "parameters": {
                    "title": "routeSchema",
                    "type": "object",
                    "properties": {
                        "next": {
                            "title": "Next",
                            "anyOf": [
                                {"enum": workers},
                            ],
                        }
                    },
                    "required": ["next"],
                },
            }
            prompt_text = f"""Your task is to select the next advisor to provide feedback on the prompt in the conversation above.
Think carefully about which advisor will provide the most valuable feedback to make improvements.
FINISH if you think your team can no longer provide valuable feedback.

The success criteria for the updated prompt are as follows:
{self.criteria}
You must use this information to inform your decision.

Select one of the below workers to provide feedback on how to improve the prompt or FINISH: 
{workers}

The details of all workers and their core principles are as follows: 
{workers_details}

Your selection process should be as follows:
1. Read the prompt as an experienced: {self.position}. Understand its content and intent.
2. Explicitly detail how you think the prompt can be improved or why you think the team can no longer provide valuable feedback.
3. If improvements are needed, determine which advisor you think will provide the most valuable feedback.
4. Submit your selection."""
            if isinstance(self.llm, ChatOpenAI):
                prompt = ChatPromptTemplate.from_messages(
                    [
                        ("system", self.system_message),
                        MessagesPlaceholder(variable_name="messages"),
                        ("system", prompt_text),
                    ]
                )
                chain = (
                    prompt
                    | self.llm.bind_functions(functions=[function_def], function_call="route")
                    | JsonOutputFunctionsParser()
                )
                result = chain.invoke({"messages": state})
                print(result["next"])
                return result["next"]
            elif isinstance(self.llm, ChatAnthropic):
                prompt = ChatPromptTemplate.from_messages(
                    [
                        ("system", self.system_message),
                        MessagesPlaceholder(variable_name="messages"),
                        ("user", prompt_text),
                    ]
                )
                try:
                    chain = (
                        prompt 
                        | self.llm.bind_tools(tools=[function_def])
                    )
                    result = chain.invoke({"messages": state})
                    if "text" in result.content[0]:
                        return result.content[1]["input"]["next"]
                    else:
                        return result.content[0]["input"]["next"]
                except Exception as e:
                    print(e)
                    return random.choice(workers)

    def update_prompt(self, state: Sequence[BaseMessage]):
        """
        Updates the prompt with the feedback from the team agent.
        """

        if len(state) == 1:
            return {"next": self.leader_decision(state)}
        
        prompt_text = f"""Your task is to improve the prompt in the conversation above in light of your core principles.
If you recieve feedback and recommendations for the prompt, respond with a revised version of your previous attempts actioning the feedback.

The success criteria for the prompt are as follows:
{self.criteria}
You will be penalized if the prompt does not meet this criteria.

Below are strict guidelines that you MUST follow if making changes to the prompt:
- DO NOT modify existing restrictions.
- DO NOT modify or remove negations.
- DO NOT add, modify or remove placeholders denoted by curly braces.
- ALWAYS treat placeholders as the actual content.
You will be penalized if you do not follow these guidelines.

Your update process should be as follows:
1. Read the prompt as an experienced: {self.position}. Understand its content and intent.
2. Think carefully about how you can implement the most recent feedback and revise the prompt.
3. Submit your revised prompt."""
        if isinstance(self.llm, ChatOpenAI):
            prompt = ChatPromptTemplate.from_messages(
                [
                    ("system", self.system_message),
                    MessagesPlaceholder(variable_name="messages"),
                    ("system", prompt_text),
                ]
            )
        elif isinstance(self.llm, ChatAnthropic):
            prompt = ChatPromptTemplate.from_messages(
                [
                    ("system", self.system_message),
                    MessagesPlaceholder(variable_name="messages"),
                    ("user", prompt_text),
                ]
            )
        chain = prompt | self.llm
        result = chain.invoke({"messages": state})
        return {"messages": [AIMessage(content=result.content, name=self.position)], "next": self.leader_decision(state)}
    
    def approval(self, state: Sequence[BaseMessage], criteria: str):
        """
        Agent to approve or reject the prompt.
        """
        function_def = {
        "name": "approval",
        "description": "Submit approval decision for the prompt.",
        "parameters": {
            "type": "object",
            "properties": {
                "team": {"type": "string", "enum": [self.position]},
                "decision": {"type": "string", "enum": ["True", "False"]},
            },
            "required": ["decision", "team"],
        },
        }
        prompt_text = f"""Your task is to decide if the prompt in the conversation above is optimal in light of your core principles.

If you think the prompt is optimal and does not require improvements in light of your core principles, return True.
If you think the prompt needs improvements in light of your core principles, return False.

Your reviewal process should be as follows:
1. Read the prompt as an experienced: {self.position}. Understand it's content and intent.
2. Determine whether the prompt is optimal in light of your core principles.
3. Submit your decision."""
        if isinstance(self.llm, ChatOpenAI):
            prompt = ChatPromptTemplate.from_messages(
                [
                    ("system", self.system_message),
                    MessagesPlaceholder(variable_name="messages"),
                    ("system", prompt_text),
                ]
            )
            chain = (
                prompt
                | self.llm.bind_functions(functions=[function_def], function_call="approval")
                | JsonOutputFunctionsParser()
            )
            result = chain.invoke({"messages": state})
            return result
        elif isinstance(self.llm, ChatAnthropic):
            prompt = ChatPromptTemplate.from_messages(
                [
                    ("system", self.system_message),
                    MessagesPlaceholder(variable_name="messages"),
                    ("user", prompt_text),
                ]
            )
            chain = (
                prompt 
                | self.llm.bind_tools(tools=[function_def])
            )
            result = chain.invoke({"messages": state})
            if "text" in result.content[0]:
                return result.content[1]["input"]
            else:
                return result.content[0]["input"]
                
    def construct_team_graph(self):
        """
        Constructs a graph of team agents based on their roles and functions.
        """

        def worker_node(state, agent):
            return agent.review_prompt(state["messages"], self.criteria)
        
        def leader_node(state):
            return self.update_prompt(state["messages"]) 
        
        # The agent state is the input to each node in the graph
        class AgentState(TypedDict):
            messages: Annotated[Sequence[BaseMessage], operator.add]
            next: str

        workflow = StateGraph(AgentState)
        for worker in self.workforce:
            # Create a node for each team agent
            node = functools.partial(worker_node, agent=worker)
            workflow.add_node(worker.position, node)
        workflow.add_node(self.position, leader_node)

        members = [worker.position for worker in self.workforce]
        for member in members:
            # We want our teams to ALWAYS "report back" to the leader when done
            workflow.add_edge(member, self.position)
        # The leader populates the "next" field in the graph state with routes to a node or finishes
        conditional_map = {k: k for k in members}
        conditional_map["FINISH"] = END
        workflow.add_conditional_edges(self.position, lambda x: x["next"], conditional_map)
        workflow.set_entry_point(self.position)

        memory = SqliteSaver.from_conn_string(":memory:")
        graph = workflow.compile(checkpointer=memory)

        return graph

In [20]:
class LeaderAgent:
    """
    LeaderAgent class defining an agent that generates and communicates worker agents to help optimise prompts.
    """

    def __init__(self, base_prompt: str, criteria: str, llm, team_leaders: List[TeamLeaderAgent]):
        self.base_prompt = base_prompt
        self.system_message = f"""You are an experienced: Lead AI Prompt Engineer. Your core principles are:
- Always pay attention to detail when designing prompts
- Always make informed decisions when designing prompts
- Always be critical when designing prompts"""
        self.criteria = criteria
        self.llm = llm
        self.team_leaders = team_leaders
        self.iterations = 0

    def reset(self):
        self.iterations = 0
    
    def run_approval(self, state: Sequence[BaseMessage]) -> List[bool]:
        """
        Run the approval process for the prompt. Run concurrently
        """
        with concurrent.futures.ThreadPoolExecutor() as executor:
            futures = [executor.submit(team.approval, state, self.criteria) for team in self.team_leaders]
            results = [future.result() for future in concurrent.futures.as_completed(futures)]
        return results
    
    def leader_decision(self, state: Sequence[BaseMessage]):
        """
        LeaderAgent to decide the next team or to finish.
        """
        self.iterations += 1
        approval_results = self.run_approval(state)
        print("Approval results:", approval_results)
        if all(approval_result['decision'] == "True" for approval_result in approval_results) or self.iterations >= 4:
            self.reset()
            return "FINISH"
        else:
            # Only ask for advise from teams that disapproved the prompt
            disapproved_teams = [approval_result['team'] for approval_result in approval_results if approval_result['decision'] == "False"]
            disapproved_teams_details = "\n".join([f"{team.position}: {team.core_principles}" for team in self.team_leaders if team.position in disapproved_teams])
            # shuffle the options to avoid positional bias
            random.shuffle(disapproved_teams)
            # options = ["FINISH"] + positions
            function_def = {
                "name": "route",
                "description": "Select the next role.",
                "parameters": {
                    "title": "routeSchema",
                    "type": "object",
                    "properties": {
                        "next": {
                            "title": "Next",
                            "anyOf": [
                                {"enum": disapproved_teams},
                            ],
                        }
                    },
                    "required": ["next"],
                },
            }
            prompt_text = f"""Your task is to choose the next team to provide feedback based on the conversation above.

The success criteria for the updated prompt are as follows:
{self.criteria}
You must use this information to inform your decision.

Select one of the below teams that disapproved of the previous prompt to provide feedback on how to improve it: 
{disapproved_teams}

The details of all disapproving teams and their core principles are as follows: 
{disapproved_teams_details}

Your selection process should be as follows:
1. Review the prompt as an experienced: Lead AI Prompt Engineer. Understand it's content and intent.
2. Explicitly detail how you think the prompt can be improved. Assume the prompt always needs improvement.
3. Determine which advisor you think will provide the most valuable feedback.
4. Submit your selection."""
            if isinstance(self.llm, ChatOpenAI):
                prompt = ChatPromptTemplate.from_messages(
                    [
                        ("system", self.system_message),
                        MessagesPlaceholder(variable_name="messages"),
                        ("system", prompt_text),
                    ]
                )
                chain = (
                    prompt
                    | self.llm.bind_functions(functions=[function_def], function_call="route")
                    | JsonOutputFunctionsParser()
                )
                result = chain.invoke({"messages": state})
                return result["next"]
            elif isinstance(self.llm, ChatAnthropic):
                prompt = ChatPromptTemplate.from_messages(
                    [
                        ("system", self.system_message),
                        MessagesPlaceholder(variable_name="messages"),
                        ("user", prompt_text),
                    ]
                )
                try: 
                    chain = (
                        prompt 
                        | self.llm.bind_tools(tools=[function_def])
                    )
                    result = chain.invoke({"messages": state})
                    print(result)
                    if "text" in result.content[0]:
                        return result.content[1]["input"]["next"]
                    else:
                        return result.content[0]["input"]["next"]
                except Exception as e:
                    print(e)
                    # return random advisor if the model is not able to generate the next advisor
                    return random.choice(disapproved_teams)

    def review_prompt(self, state: Sequence[BaseMessage]) -> str:
        """
        Updates the prompt with the feedback from the team agent.
        """

        if len(state) == 1:
            return {"next": self.leader_decision(state)}
        
        prompt_text = f"""Your task is to check the prompt in the conversation above meets the success criteria and adheres to the strict guidelines.
If the success criteria are not met, or the strict guidlines not followed, provide a carefully revised version of the prompt.

The success criteria for the prompt are as follows:
{self.criteria}

The strict guidelines for the prompt are as follows:
- DO NOT modify existing restrictions.
- DO NOT modify or remove negations.
- DO NOT add, modify or remove placeholders denoted by curly braces. If you wish to use curly braces in your response, use double curly braces to avoid confusion with placeholders.
- ALWAYS treat placeholders as the actual content.

Your reviewal process should be as follows:
1. Review the prompt as an experienced: Lead AI Prompt Engineer. Understand it's content and intent.
2. Explcitly go through each success criteria and ensure the prompt meets them. If not, revise the prompt to meet them.
3. Explicitly go through each guideline and ensure changes made adhere to them. If not, revise the prompt to adhere to them.
4. If the prompt meets the success criteria and adheres to the strict guidelines, submit the prompt.
"""
        if isinstance(self.llm, ChatOpenAI):
            prompt = ChatPromptTemplate.from_messages(
                [
                    ("system", self.system_message),
                    MessagesPlaceholder(variable_name="messages"),
                    ("system", prompt_text),
                ]
            )
        elif isinstance(self.llm, ChatAnthropic):
            prompt = ChatPromptTemplate.from_messages(
                [
                    ("system", self.system_message),
                    MessagesPlaceholder(variable_name="messages"),
                    ("user", prompt_text),
                ]
            )
        chain = prompt | self.llm
        result = chain.invoke({"messages": state})
        return {"messages": [HumanMessage(content=result.content, name="Leader")], "next": self.leader_decision(state)}

    def construct_team_graph(self):
        """
        Constructs a graph of team agents based on their roles and functions.
        """

        # The agent state is the input to each node in the graph
        class AgentState(TypedDict):
            messages: Annotated[Sequence[BaseMessage], operator.add]
            next: str

        def enter_chain(message: str):
            results = {
                "messages": [HumanMessage(content=message)],
            }
            return results
        
        def get_last_message(state: AgentState) -> str:
            return state["messages"][-1].content

        def join_graph(response: dict):
            return {"messages": [response["messages"][-1]]}

        def leader_node(state):
            return self.review_prompt(state["messages"])

        workflow = StateGraph(AgentState)
        for team in self.team_leaders:
            # Create a node for each team agent
            graph = team.construct_team_graph()
            chain = enter_chain | graph
            workflow.add_node(team.position, get_last_message | chain | join_graph)
        workflow.add_node("Leader", leader_node)

        members = [team.position for team in self.team_leaders]
        for member in members:
            # We want our teams to ALWAYS "report back" to the leader when done
            workflow.add_edge(member, "Leader")
        # The leader populates the "next" field in the graph state with routes to a node or finishes
        conditional_map = {k: k for k in members}
        conditional_map["FINISH"] = END
        workflow.add_conditional_edges("Leader", lambda x: x["next"], conditional_map)
        workflow.set_entry_point("Leader")

        memory = SqliteSaver.from_conn_string(":memory:")
        graph = workflow.compile(checkpointer=memory)

        return graph
    
    def optimise_prompt(self):
        """
        Optimises a prompt by invoking a graph of worker agents.
        """
        # Initial state
        initial_state = {
            "messages": [HumanMessage(content=f"{self.base_prompt}", name="User")],
        }

        # Construct the graph
        graph = self.construct_team_graph()
        # display(Image(graph.get_graph().draw_mermaid_png()))

        n = random.randint(0, 1000)
        config = {
            "configurable": {"thread_id": str(n)},
            "recursion_limit": 50,
            }    

        # Run the graph
        for s in graph.stream(
            initial_state,
            config,
            stream_mode="values",
            ):
            if "__end__" not in s:
                if len(s["messages"]) > 1:
                    s["messages"][-1].pretty_print()
                continue

        def message_to_dict(obj):
            if isinstance(obj, HumanMessage) or isinstance(obj, AIMessage):
                return {obj.name: obj.content}
            raise TypeError(f'Object of type {obj.__class__.__name__} is not JSON serializable')

        if type(self.llm) == ChatOpenAI:
            model = self.llm.model_name
        else:
            model = self.llm.model
        temp = int(self.llm.temperature)
        path = f"/Users/iwatson/Documents/Research Project/prompt-optimisation/src/conversations/{model}/conversations_hierarchcial_{temp}.json"
        if not os.path.exists(path):
            with open(path, "w") as f:
                json.dump([], f)
        
        with open(path, "r") as f:
            # write messages to json file
            data = json.load(f)
            # get the current key number then increment it
            key = len(data)
            data.append({key: json.dumps(s, default=message_to_dict)})
            
        with open(path, "w") as f:
            json.dump(data, f, indent=4)
                
        return s

In [21]:
# llm = ChatAnthropic(temperature=0, model="claude-3-haiku-20240307")
# llm = ChatAnthropic(temperature=0, model="claude-3-5-sonnet-20240620")
# llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")
# llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")
llm = ChatOpenAI(temperature=0, model="gpt-4o")
# llm = ChatOllama(temperature=1, model="mistral:v0.3")
# llm = ChatOllama(temperature=0, model="llama3.1")

In [24]:
from prompts import gsm8k, human_eval, sst2

human_eval_baseline = human_eval.get_baseline_prompt()
human_eval_criteria = human_eval.get_criteria()

gsm8k_baseline = gsm8k.get_baseline_prompt()
gsm8k_criteria = gsm8k.get_criteria()

sst2_baseline_prompt = sst2.get_baseline_prompt()
sst2_criteria = sst2.get_criteria()

In [25]:
from agent_suite import PromptDesignAgents, HumanEvalAgents, GSM8kAgents, SST2Agents

prompt_design_agents = PromptDesignAgents()

style_and_structure_expert = WorkerAgent("Style_and_Structure_Expert", CorePrinciples(prompt_design_agents.get_style_and_structure_principles()), llm)
conciseness_and_clarity_expert = WorkerAgent("Conciseness_and_Clarity_Expert", CorePrinciples(prompt_design_agents.get_conciseness_and_clarity_principles()), llm)
contextual_relevance_expert = WorkerAgent("Contextual_Relevance_Expert", CorePrinciples(prompt_design_agents.get_contextual_relevance_principles()), llm)
task_alignment_expert = WorkerAgent("Task_Alignment_Expert", CorePrinciples(prompt_design_agents.get_task_alignment_principles()), llm)
example_demonstration_expert = WorkerAgent("Example_Demonstration_Expert", CorePrinciples(prompt_design_agents.get_example_demonstration_principles()), llm)
incremental_prompting_expert = WorkerAgent("Incremental_Prompting_Expert", CorePrinciples(prompt_design_agents.get_incremental_prompting_principles()), llm)

human_eval_agents = HumanEvalAgents()

code_reviewer = WorkerAgent("Code_Reviewer", CorePrinciples(human_eval_agents.get_code_reviewer_principles()), llm)
software_engineer = WorkerAgent("Software_Engineer", CorePrinciples(human_eval_agents.get_software_engineering_principles()), llm)
software_architect = WorkerAgent("Software_Architect", CorePrinciples(human_eval_agents.get_software_architecture_principles()), llm)

gsm8k_agents = GSM8kAgents()

mathematician = WorkerAgent("Mathematician", CorePrinciples(gsm8k_agents.get_mathematician_principles()), llm)
word_problem_solver = WorkerAgent("Word_Problem_Solver", CorePrinciples(gsm8k_agents.get_word_problem_solver_principles()), llm)

sst2_agents = SST2Agents()

graded_sentiment_analyst = WorkerAgent("Graded_Sentiment_Analyst", CorePrinciples(sst2_agents.get_graded_sentiment_analyst_principles()), llm)
emotive_sentiment_analyst = WorkerAgent("Emotive_Sentiment_Analyst", CorePrinciples(sst2_agents.get_emotive_sentiment_analyst_principles()), llm)
aspect_based_sentiment_analyst = WorkerAgent("Aspect_Based_Sentiment_Analyst", CorePrinciples(sst2_agents.get_aspect_based_sentiment_analyst_principles()), llm)

In [32]:
dataset = "human_eval"
if dataset == "human_eval":
    baseline_prompt = human_eval_baseline
    criteria = human_eval_criteria
    domain_experts = [software_engineer, software_architect, code_reviewer]
    domain_leader = "Lead_Software_Engineer"
    domain_principles = CorePrinciples([human_eval_agents.get_lead_software_engineer_principles()])
elif dataset == "gsm8k":
    baseline_prompt = gsm8k_baseline
    criteria = gsm8k_criteria
    domain_experts = [mathematician, word_problem_solver]
    domain_leader = "Lead_Mathematician"
    domain_principles = CorePrinciples([gsm8k_agents.get_lead_mathematician_principles()])
elif dataset == "sst2":
    baseline_prompt = sst2_baseline_prompt
    criteria = sst2_criteria
    domain_experts = [graded_sentiment_analyst, emotive_sentiment_analyst, aspect_based_sentiment_analyst]
    domain_leader = "Lead_Sentiment_Analyst"
    domain_principles = CorePrinciples([sst2_agents.get_lead_sentiment_analyst_principles()])

In [33]:
prompt_design_team = TeamLeaderAgent(
    "Lead_Prompt_Writer",
    CorePrinciples([prompt_design_agents.get_lead_prompt_writer_principles()]),
    baseline_prompt,
    criteria,
    workforce=[
        style_and_structure_expert,
        conciseness_and_clarity_expert,
        contextual_relevance_expert,
        task_alignment_expert,
        example_demonstration_expert,
        incremental_prompting_expert,
    ],
    llm=llm
)
domain_team = TeamLeaderAgent(
    domain_leader,
    domain_principles,
    baseline_prompt,
    criteria,
    workforce=domain_experts,
    llm=llm
)

leader_agent = LeaderAgent(
    base_prompt=baseline_prompt,
    criteria=criteria,
    team_leaders=[prompt_design_team, domain_team],
    llm=llm
)

# leader_agent.generate_teams()
# print teams and members
print("Teams and members:")
for team_leader in leader_agent.team_leaders:
    print(f"{team_leader.position} team:")
    for worker in team_leader.workforce:
        print(f"Position: {worker.position}, Core Principles: \n{worker.core_principles}")
    print("----")

Teams and members:
Lead_Prompt_Writer team:
Position: Style_and_Structure_Expert, Core Principles: 
- Always structure prompts logically for the task
- Always use a style and tone in prompts that is appropriate for the task
- Always assign a role to the language model that is relevant to the task
Position: Conciseness_and_Clarity_Expert, Core Principles: 
- Always write clear and concise prompts
- Always use simple and direct language in prompts
- Always avoid ambiguity in prompts
Position: Contextual_Relevance_Expert, Core Principles: 
- Always provide context to help the model understand the task
- Always write prompts informed by the context of the task
- Always design contextually relevant roles for the language model
Position: Task_Alignment_Expert, Core Principles: 
- Always write prompts that align with the task criteria
- Always tailor instructions to the task to guide the model
- Always make the task abundantly clear to the model in the prompt
Position: Example_Demonstration_E

In [34]:
import time

times = []
for _ in range(1):
    start = time.time()
    result = leader_agent.optimise_prompt()
    end = time.time()
    times.append(end - start)
    print(f"Time taken: {end - start}")
    result["messages"][-2].pretty_print()
    print("--------------------")

Approval results: [{'team': 'Lead_Prompt_Writer', 'decision': 'False'}, {'team': 'Lead_Software_Engineer', 'decision': 'False'}]
Task_Alignment_Expert
Conciseness_and_Clarity_Expert
FINISH
================================== Ai Message ==================================
Name: Lead_Prompt_Writer

### Revised Prompt

Complete the function below using its signature and docstring. Ensure it is fully functional and handles edge cases.

Function signature and docstring:
```python
{content}
```

Output the completed function as:
```python
<your answer>
```
Approval results: [{'team': 'Lead_Software_Engineer', 'decision': 'False'}, {'team': 'Lead_Prompt_Writer', 'decision': 'False'}]
================================ Human Message =================================
Name: Leader

### Review of the Prompt

#### Success Criteria Check:
1. **Instructs the LLM to complete a function based on its signature and docstring:**
   - Yes, the prompt clearly instructs the LLM to complete the function using it

In [35]:
result["messages"][-1].pretty_print()

================================ Human Message =================================
Name: Leader

### Review of the Prompt

#### Success Criteria Check:
1. **Instructs the LLM to complete a function based on its signature and docstring:**
   - Yes, the prompt clearly instructs the LLM to complete the function using its signature and docstring.
   
2. **Includes the content placeholder:**
   - Yes, the placeholder `{content}` is included in the prompt.
   
3. **Instructs the model to output the answer at the end as ```python <your answer> ``` :**
   - Yes, the prompt instructs the model to output the answer in the specified format.

#### Guidelines Check:
1. **DO NOT modify existing restrictions:**
   - No existing restrictions were modified.
   
2. **DO NOT modify or remove negations:**
   - No negations were modified or removed.
   
3. **DO NOT add, modify or remove placeholders denoted by curly braces:**
   - The placeholder `{content}` was not modified or removed.
   
4. **ALWAYS treat

In [ ]:
print("Max time: ", max(times))
print("Min time: ", min(times))
print("Average time: ", sum(times) / len(times))